# Reconhecimento de flores e árvores

## Autora
* Isabella Carneiro Godinho de Morais Sá
* Matrícula: 18/0019066
* Github: isabellacgmsa

## Objetivo
O propósito do presente artigo é demonstrar o conhecimento adquirido através da terceira lição, que fala sobre diferentes modelos de treinamento.

## Modelos
Até o momento estava utilizando o Resnet, porém na terceira lição há a indagação de qual seria o melhor modelo tendo em vista as particularidades de cada projeto. Para chegar a uma conclusão, é necessário testá-los.

## Passo 1 
A instalação das dependências é um passo fundamental para garantir que o processo possa ser executado com sucesso e que os resultados sejam precisos e confiáveis. Além disso, a escolha das bibliotecas e ferramentas adequadas é essencial para garantir que o processo seja executado de maneira eficiente e eficaz, maximizando os recursos disponíveis e minimizando o tempo necessário para a execução do processo.
Assim sendo, o primeiro passo que foi tomado para iniciar o projeto foi a instalação das dependências necessárias para que o processo pudesse ser executado de maneira adequada.
Para tanto, foram utilizados os seguintes comandos para a instalação das dependências necessárias:

```pip install -U duckduckgo_search```

```pip install fastai"```

Vale ressaltar que, antes de iniciar o processo, é necessário fazer a instalação do PyTorch, que é uma biblioteca de aprendizado de máquina de código aberto amplamente utilizada para tarefas como visão computacional e processamento de linguagem natural.

## Passo 2 
Para fazer a instalação das imagens foi utilizado o ddg_images, como mostra o código a seguir:  
```
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term: str, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')
    
```

O primeiro teste será feito baixando apenas uma imagem de flor:

```
urls = search_images('flower', max_images=1)
urls[2]
```

O download será realizado utilizando a biblioteca fastdownload:

```
from fastdownload import download_url
dest = 'oneflower.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)
```

![texto](https://s3.static.brasilescola.uol.com.br/be/2020/12/girassol.jpg)

Ao constatar que está funcionando todo o processo é possível fazer o download de dados mais numerosos:
```
searches = 'tree', 'flower'
path = Path('tree_flower')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} tree'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} flower'))
    sleep(10)
    resize_images(path/o, max_size=400, dest= path/o)
```

Foi utilizado o sleep para que não haja problemas no tráfego de informações.

Essas imagens serão salvas em uma pasta cujo caminho está definido no path.

Antes de iniciar o treinamento do modelo de fato é necessário garantir que não haja imagens com erro para evitar futuros problemas:

```
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)
```

Para treinar modelos de aprendizado de máquina com conjuntos de dados volumosos, é necessário dividir os dados em lotes menores para evitar sobrecarregar a memória RAM do computador. O Dataloader é uma classe que oferece essa funcionalidade de forma eficiente. Ele carrega e processa os dados em lotes menores, tornando o processo de treinamento escalável.

O fastai é um framework de aprendizado de máquina que permite o uso do datablock, uma ferramenta que facilita a personalização do processamento e organização dos dados em lotes para treinamento do modelo. Com o datablock, é possível definir etapas como leitura, pré-processamento e organização dos dados de forma mais simples. Isso permite que os usuários criem fluxos de trabalho personalizados e experimentem diferentes abordagens para melhorar a precisão do modelo.

```
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)
```
![texto](https://i.ibb.co/T4XzYk1/flores.png)

Foi especificado que há o ImageBlock, que é uma classe do fastai que permite a criação de blocos para processar as imagens no modelo, e o CategoryBlock que cria blocos para processar variáveis categóricas, nesse caso árvore ou flor. Nesse trecho também especifica o tamanho que as imagens terão, já que precisa ser o mesmo para todas.

### Treinamentos
### ResNet18
O ResNet18 é um modelo de rede neural convolucional que possui 18 camadas e é considerado uma versão mais compacta do ResNet original. Isso o torna mais fácil de ser treinado e executado em dispositivos com recursos limitados. Ele foi pré-treinado em grandes conjuntos de dados de imagens, como o ImageNet, o que permite sua utilização em tarefas de classificação de imagem com alta precisão.

Em síntese, o ResNet18 utiliza blocos residuais para permitir que informações sejam transmitidas pelas camadas de forma mais eficiente, permitindo a construção de redes mais profundas. Devido à sua eficácia e eficiência, é uma escolha popular para tarefas de classificação de imagem.
```
learn = vision_learner(dls, resnet18, metrics=error_rate).to_fp16()
learn.fine_tune(8)

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
```
![imagem](https://i.ibb.co/j6VNmbC/matriz-Copia.png)


### Outro
A arquitetura do RegNetY é composta por um módulo de construção de bloco que é replicado várias vezes para formar a rede. Cada bloco é constituído por um conjunto de camadas, que incluem convoluções, normalização em lote e ativação, e possui uma conexão residual para permitir a passagem de informações pela rede de maneira mais eficiente. O número de blocos e a largura de banda são definidos por meio de uma pesquisa automática de arquitetura baseada na RegNetX, uma arquitetura anterior desenvolvida pela equipe do FAIR.

Em comparação com outras arquiteturas populares, como ResNet e EfficientNet, o RegNetY é capaz de atingir resultados de precisão de ponta em conjuntos de dados de referência, como o ImageNet, utilizando uma arquitetura mais enxuta e eficiente.
```
learn2 = vision_learner(dls, 'regnety_008', metrics=error_rate).to_fp16()
learn2.fine_tune(8)

interp = ClassificationInterpretation.from_learner(learn2)
interp.plot_confusion_matrix()
```
![imagem](https://i.ibb.co/PG2hMjz/matriz.png)



### Outro
A arquitetura LeViT consiste em um conjunto de blocos de convolução intercalados com blocos de atenção, uma característica comum da arquitetura Transformer. Após cada bloco de convolução, há um bloco de atenção para permitir que a rede capture informações espaciais e relacionais em diferentes níveis de abstração.

O objetivo da LeViT é combinar o desempenho de arquiteturas baseadas em Transformer com a eficiência de modelos convolucionais, tornando-a uma opção adequada para aplicações em tempo real em dispositivos com recursos limitados. De acordo com a equipe do FAIR, a LeViT alcançou resultados competitivos em vários conjuntos de dados de visão computacional, incluindo o ImageNet, enquanto apresenta uma vantagem significativa em termos de eficiência em relação a outros modelos baseados em Transformer.
```
learn3 = vision_learner(dls, 'levit_256', metrics=error_rate).to_fp16()
learn3.fine_tune(8)

interp = ClassificationInterpretation.from_learner(learn3)
interp.plot_confusion_matrix()
```
![imagem](https://i.ibb.co/M2PrQyf/matriz-Copia-2.png)


## Conclusão
Em resumo, os modelos ResNet, RegNetY e LeViT têm em comum a mesma finalidade de resolver tarefas de visão computacional, como classificação de imagem. No entanto, esses modelos apresentam diferenças significativas em sua arquitetura, desempenho e eficiência, tornando-os mais adequados para diferentes tipos de aplicações e tarefas específicas.
Por exemplo, o ResNet18 é um modelo mais simples e menor que o ResNet original, o que o torna mais rápido para treinar e mais leve para ser executado em dispositivos com recursos limitados. É uma opção popular para tarefas de classificação de imagem devido à sua eficácia e eficiência.
Por outro lado, o RegNetY é capaz de atingir resultados de precisão de ponta em conjunto de dados de referência, como o ImageNet, com uma arquitetura mais enxuta e eficiente do que outras arquiteturas populares, como ResNet e EfficientNet. Isso ocorre porque o RegNetY é construído com base em uma pesquisa automática de arquitetura baseada em RegNetX, que permite a determinação do número de blocos e da largura de banda mais adequados.
Por fim, a LeViT combina o desempenho de arquiteturas baseadas em Transformer com a eficiência de modelos convolucionais, tornando-a uma opção viável para aplicações em tempo real em dispositivos com recursos limitados. Além disso, a LeViT apresenta vantagens significativas em termos de eficiência em relação a outros modelos baseados em Transformer, tornando-a uma opção atraente para aplicativos com restrições de recursos.
Ao escolher um modelo, deve se levar em conta a acurácia e o tempo, avaliando qual fator tem mais peso dentro do objetivo da aplicação a ser feita.



